In [1]:
#import all the relevent libraries

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import json
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [2]:
#read the json file
df_ny = pd.read_json('https://data.cityofnewyork.us/resource/qiz3-axqb.json')

In [3]:
#display first five entries of the data frame  source file
df_ny.head()

,accident_date,accident_time,borough,collision_id,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,latitude,location,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,off_street_name,on_street_name,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code
0,2019-04-18T00:00:00.000,2019-11-29 20:57:00,BRONX,4117220,Traffic Control Disregarded,Unspecified,Unspecified,NaN,NaN,NaN,40.813652,"{'type': 'Point', 'coordinates': [-73.906975, ...",-73.906975,0,0,2,0,0,0,2,0,EAST 150 STREET,WALES AVENUE,Station Wagon/Sport Utility Vehicle,Sedan,Sedan,NaN,NaN,10455.0
1,2019-05-10T00:00:00.000,2019-11-29 08:58:00,MANHATTAN,4129801,Passing Too Closely,Unspecified,NaN,NaN,NaN,33 WEST 42 STREET,40.754000,"{'type': 'Point', 'coordinates': [-73.98211, 4...",-73.982110,0,0,0,0,0,0,0,0,NaN,NaN,Bus,Sedan,NaN,NaN,NaN,10036.0
2,2019-04-29T00:00:00.000,2019-11-29 18:30:00,BRONX,4123153,Unspecified,Unspecified,NaN,NaN,NaN,NaN,40.906860,"{'type': 'Point', 'coordinates': [-73.904106, ...",-73.904106,0,0,0,0,0,0,0,0,RIVERDALE AVENUE,WEST 259 STREET,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,10471.0
3,2019-05-12T00:00:00.000,2019-11-29 18:00:00,NaN,4130226,Unspecified,Unspecified,NaN,NaN,NaN,NaN,40.664160,"{'type': 'Point', 'coordinates': [-73.990524, ...",-73.990524,0,0,1,0,0,0,1,0,NaN,PROSPECT AVENUE,Sedan,Sedan,NaN,NaN,NaN,NaN
4,2019-04-18T00:00:00.000,2019-11-29 13:00:00,NaN,4117578,Unspecified,NaN,NaN,NaN,NaN,NaN,40.774235,"{'type': 'Point', 'coordinates': [-73.795494, ...",-73.795494,0,0,0,0,0,0,0,0,NaN,169 STREET,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN


In [4]:
#get the releent object types of the data frame
df_ny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
accident_date                    1000 non-null object
accident_time                    1000 non-null datetime64[ns]
borough                          647 non-null object
collision_id                     1000 non-null int64
contributing_factor_vehicle_1    994 non-null object
contributing_factor_vehicle_2    855 non-null object
contributing_factor_vehicle_3    58 non-null object
contributing_factor_vehicle_4    13 non-null object
contributing_factor_vehicle_5    3 non-null object
cross_street_name                234 non-null object
latitude                         914 non-null float64
location                         914 non-null object
longitude                        914 non-null float64
number_of_cyclist_injured        1000 non-null int64
number_of_cyclist_killed         1000 non-null int64
number_of_motorist_injured       1000 non-null int64
number_of_motorist_killed        1000 

In [5]:
df_ny.shape

(1000, 29)

In [6]:
df_ny.dropna()

,accident_date,accident_time,borough,collision_id,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,latitude,location,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,off_street_name,on_street_name,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code


In [7]:
#read the first five entires of collision data
df_ny.head()

,accident_date,accident_time,borough,collision_id,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,latitude,location,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,off_street_name,on_street_name,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code
0,2019-04-18T00:00:00.000,2019-11-29 20:57:00,BRONX,4117220,Traffic Control Disregarded,Unspecified,Unspecified,NaN,NaN,NaN,40.813652,"{'type': 'Point', 'coordinates': [-73.906975, ...",-73.906975,0,0,2,0,0,0,2,0,EAST 150 STREET,WALES AVENUE,Station Wagon/Sport Utility Vehicle,Sedan,Sedan,NaN,NaN,10455.0
1,2019-05-10T00:00:00.000,2019-11-29 08:58:00,MANHATTAN,4129801,Passing Too Closely,Unspecified,NaN,NaN,NaN,33 WEST 42 STREET,40.754000,"{'type': 'Point', 'coordinates': [-73.98211, 4...",-73.982110,0,0,0,0,0,0,0,0,NaN,NaN,Bus,Sedan,NaN,NaN,NaN,10036.0
2,2019-04-29T00:00:00.000,2019-11-29 18:30:00,BRONX,4123153,Unspecified,Unspecified,NaN,NaN,NaN,NaN,40.906860,"{'type': 'Point', 'coordinates': [-73.904106, ...",-73.904106,0,0,0,0,0,0,0,0,RIVERDALE AVENUE,WEST 259 STREET,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,10471.0
3,2019-05-12T00:00:00.000,2019-11-29 18:00:00,NaN,4130226,Unspecified,Unspecified,NaN,NaN,NaN,NaN,40.664160,"{'type': 'Point', 'coordinates': [-73.990524, ...",-73.990524,0,0,1,0,0,0,1,0,NaN,PROSPECT AVENUE,Sedan,Sedan,NaN,NaN,NaN,NaN
4,2019-04-18T00:00:00.000,2019-11-29 13:00:00,NaN,4117578,Unspecified,NaN,NaN,NaN,NaN,NaN,40.774235,"{'type': 'Point', 'coordinates': [-73.795494, ...",-73.795494,0,0,0,0,0,0,0,0,NaN,169 STREET,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN


In [8]:
#create the corresponding subset 
df_ny.dropna(subset=['borough'])

,accident_date,accident_time,borough,collision_id,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,latitude,location,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,off_street_name,on_street_name,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code
0,2019-04-18T00:00:00.000,2019-11-29 20:57:00,BRONX,4117220,Traffic Control Disregarded,Unspecified,Unspecified,NaN,NaN,NaN,40.813652,"{'type': 'Point', 'coordinates': [-73.906975, ...",-73.906975,0,0,2,0,0,0,2,0,EAST 150 STREET,WALES AVENUE,Station Wagon/Sport Utility Vehicle,Sedan,Sedan,NaN,NaN,10455.0
1,2019-05-10T00:00:00.000,2019-11-29 08:58:00,MANHATTAN,4129801,Passing Too Closely,Unspecified,NaN,NaN,NaN,33 WEST 42 STREET,40.754000,"{'type': 'Point', 'coordinates': [-73.98211, 4...",-73.982110,0,0,0,0,0,0,0,0,NaN,NaN,Bus,Sedan,NaN,NaN,NaN,10036.0
2,2019-04-29T00:00:00.000,2019-11-29 18:30:00,BRONX,4123153,Unspecified,Unspecified,NaN,NaN,NaN,NaN,40.906860,"{'type': 'Point', 'coordinates': [-73.904106, ...",-73.904106,0,0,0,0,0,0,0,0,RIVERDALE AVENUE,WEST 259 STREET,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,10471.0
6,2019-05-13T00:00:00.000,2019-11-29 18:20:00,BROOKLYN,4131405,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,NaN,40.694120,"{'type': 'Point', 'coordinates': [-73.99463, 4...",-73.994630,0,0,0,0,0,0,0,0,HENRY STREET,REMSEN STREET,Sedan,Sedan,NaN,NaN,NaN,11201.0
7,2019-04-24T00:00:00.000,2019-11-29 10:26:00,MANHATTAN,4119970,Other Vehicular,Other Vehicular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,WEST 52 STREET,7 AVENUE,Taxi,Sedan,NaN,NaN,NaN,10019.0
8,2019-04-15T00:00:00.000,2019-11-29 15:00:00,BRONX,4115841,View Obstructed/Limited,Unspecified,NaN,NaN,NaN,NaN,40.879790,"{'type': 'Point', 'coordinates': [-73.86493, 4...",-73.864930,0,0,0,0,0,0,0,0,WHITE PLAINS ROAD,EAST 214 STREET,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,10467.0
9,2019-05-04T00:00:00.000,2019-11-29 14:10:00,BROOKLYN,4126605,Unspecified,Unspecified,NaN,NaN,NaN,NaN,40.619694,"{'type': 'Point', 'coordinates': [-73.914856, ...",-73.914856,0,0,0,0,0,0,0,0,EAST 65 STREET,VETERANS AVENUE,Sedan,Sedan,NaN,NaN,NaN,11234.0
10,2019-05-03T00:00:00.000,2019-11-29 00:15:00,BROOKLYN,4124590,Following Too Closely,Unspecified,NaN,NaN,NaN,NaN,40.675735,"{'type': 'Point', 'coordinates': [-73.89686, 4...",-73.896860,0,0,0,0,0,0,0,0,PENNSYLVANIA AVENUE,ATLANTIC AVENUE,Sedan,Sedan,NaN,NaN,NaN,11207.0
11,2019-05-01T00:00:00.000,2019-11-29 16:52:00,BRONX,4124025,Unsafe Lane Changing,Unsafe Lane Changing,NaN,NaN,NaN,NaN,40.862534,"{'type': 'Point', 'coordinates': [-73.89708, 4...",-73.897080,0,0,0,0,0,0,0,0,GRAND CONCOURSE,EAST FORDHAM ROAD,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,10468.0
13,2019-04-17T00:00:00.000,2019-11-29 09:20:00,BROOKLYN,4116679,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,40.586445,"{'type': 'Point', 'coordinates': [-73.98838, 4...",-73.988380,0,0,0,0,0,0,0,0,28 AVENUE,CROPSEY AVENUE,Dump,Sedan,NaN,NaN,NaN,11214.0


In [9]:
#find the locations with the no. of collision of occurance
df_ny['borough'].value_counts()

BROOKLYN         215
QUEENS           181
BRONX            120
MANHATTAN        118
STATEN ISLAND     13
Name: borough, dtype: int64

In [10]:
#get the relevent information with contibuting factor 1
df1= df_ny[['borough', 'contributing_factor_vehicle_1']]

In [11]:
df1.dropna(subset=['borough'])

,borough,contributing_factor_vehicle_1
0,BRONX,Traffic Control Disregarded
1,MANHATTAN,Passing Too Closely
2,BRONX,Unspecified
6,BROOKLYN,Driver Inattention/Distraction
7,MANHATTAN,Other Vehicular
8,BRONX,View Obstructed/Limited
9,BROOKLYN,Unspecified
10,BROOKLYN,Following Too Closely
11,BRONX,Unsafe Lane Changing
13,BROOKLYN,Driver Inattention/Distraction


In [12]:
#Notice that the driven inattention and distration is the major cause of collision
df1['contributing_factor_vehicle_1'].value_counts()

Driver Inattention/Distraction                           238
Unspecified                                              222
Following Too Closely                                     94
Failure to Yield Right-of-Way                             71
Passing Too Closely                                       49
Passing or Lane Usage Improper                            47
Backing Unsafely                                          42
Other Vehicular                                           32
Reaction to Uninvolved Vehicle                            29
Unsafe Lane Changing                                      28
Turning Improperly                                        26
Traffic Control Disregarded                               22
Unsafe Speed                                              18
Driver Inexperience                                       17
Alcohol Involvement                                       10
Pavement Slippery                                          8
View Obstructed/Limited 

In [13]:
df2=df1.groupby(['borough','contributing_factor_vehicle_1']).size()

In [14]:
print(df2)

borough        contributing_factor_vehicle_1                        
BRONX          Alcohol Involvement                                       1
               Backing Unsafely                                          5
               Driver Inattention/Distraction                           14
               Driver Inexperience                                       2
               Failure to Yield Right-of-Way                             9
               Fell Asleep                                               1
               Following Too Closely                                     4
               Other Vehicular                                           8
               Outside Car Distraction                                   2
               Oversized Vehicle                                         1
               Passenger Distraction                                     1
               Passing Too Closely                                       6
               Passing or Lane 

In [17]:
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim 
import requests

In [18]:
#provide the foursquare details
CLIENT_ID = 'DNJUIKPDJK4HDBDQPADRIJKGYD2AUTPKJ1I41OJUFGG2FHEV' # your Foursquare ID
CLIENT_SECRET = '42DLW0Q0QDH3PTVCLZE50MZWXZQ0NH0QSZF01REAANB3CGYV' # your Foursquare Secret
VERSION = '20191129'
LIMIT = 100

In [19]:
#import clustering and geolocation packages and libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')

Libraries imported.


In [32]:
#load the data which contains geo info of NY
List_url='https://geo.nyu.edu/catalog/nyu_2451_34572'
source = requests.get(List_url).text

In [35]:
with open('nyu-2451-34572-geojson.json') as json_data:
    df_newyork = json.load(json_data)
   

In [36]:
df_newyork = df_newyork['features']

In [37]:
#observe the various features 
df_newyork[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [38]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
newyork_data = pd.DataFrame(columns=column_names)

In [39]:
len(df_newyork)

306

In [40]:
# let us populate it
for data in df_newyork:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    newyork_data = newyork_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [41]:
newyork_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [42]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(newyork_data['Borough'].unique()),
        newyork_data.shape[0]
    )
)

#how many neighborhood each borough has
df_groups = newyork_data.groupby (['Borough']).count()
df_groups

The dataframe has 5 boroughs and 306 neighborhoods.


,Neighborhood,Latitude,Longitude
Borough,,,
Bronx,52,52,52
Brooklyn,70,70,70
Manhattan,40,40,40
Queens,81,81,81
Staten Island,63,63,63


In [43]:
# Use geopy library to get the latitude and longitude values of New York City.
address = 'New York City, NY'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

C:\Users\sheela.ganesh.t\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [52]:
manhattan_data = newyork_data[newyork_data['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [51]:
bronx_data = newyork_data[newyork_data['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [48]:

queens_data = newyork_data[newyork_data['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [49]:

StatenIsland_data = newyork_data[newyork_data['Borough'] == 'Staten Island'].reset_index(drop=True)
StatenIsland_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182
